### ***Extraction***

In [7]:
import camelot
import pandas as pd

def traiter_pdf_simple(pdf_path):
    
    # Extraction
    tables = camelot.read_pdf(pdf_path, pages="all", flavor="lattice")
    print(f"{len(tables)} tableaux extraits")
    
    # Fusion
    dfs = [table.df for table in tables]
    dataset = pd.concat(dfs, ignore_index=True)
    
    return dataset

# Utilisation en une ligne
df = traiter_pdf_simple("E:\\Projet_artefact\\EDAN_2025_RESULTAT_NATIONAL_DETAILS.pdf")
print(f"Dataset final: {df.shape}")
df.head(1000)

e:\Projet_artefact\chalenge_arte\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


35 tableaux extraits
Dataset final: (1196, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,REGI\nCIRCONSCRIPTION\nON,,,NB BV,INSCRITS,VOTANTS,TAUX DE \nPART.,BULL. \nNULS,SUF. \nEXPRIMES,BULL. BLANCS,,GROUPEMENTS / PARTIS \nPOLITIQUES,CANDIDATS / LISTES DE CANDIDATS,SCORES,%,
1,,,,,,,,,,NOMBRE,%,,,,,
2,TOTAL,,,25 338,8 597 092,3 012 094,"35,04%",68 525,2 943 569,29 578,"1,00%",,,2 913 991,,
3,AGNEBY-TIASSA,001,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, \nL...",144,52 106,14070,"27,00%",388,13 682,76,"0,56%",INDEPENDANT,KOTO EHOU SOPIE,547,"4,00%",
4,,,,,,,,,,,,RHDP,KOFFI AKA CHARLES,9 078,"66,35%",ELU(E)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,,,,,,,,,,,,INDEPENDANT,AKA AKESSE KOUADIO VINCENT,231,"2,27%",
996,,,,,,,,,,,,RHDP,COULIBALY ADAMA,6 793,"66,84%",ELU(E)
997,,,,,,,,,,,,PDCI-RDA,KONAN KOUAKOU RENE,2 513,"24,73%",
998,REGI\nON,CIRCONSCRIPTION,,NB BV,INSCRITS,VOTANTS,TAUX DE \nPART.,BULL. \nNULS,SUF. \nEXPRIMES,BULL. BLANCS,,GROUPEMENTS / PARTIS \nPOLITIQUES,CANDIDATS / LISTES DE CANDIDATS,SCORES,%,


In [8]:
def nettoyer(df):
    df_copy = df.copy()

    # Renommage colonnes
    df_copy.columns = [
    'region', 'N° REGION', 'circonscription', 'nombre_bureaux_vote', 'inscrits', 
    'votants', 'taux_de_participation', 'bulletins_nuls', 'suffrages_exprimes', 
    'bulletins_blancs', 'pourcentage_blancs', 'parti', 
    'candidat', 'voix', 'pourcentage_voix', 'elu']
    
    # Nettoyage
    # Trouver colonne nombre_bureaux_vote
    col_nb_bv = "nombre_bureaux_vote"
    # Collecter indices
    idx_supprimer = []
    i = 0
    
    while i < len(df_copy):
        if "NB BV" in str(df_copy.at[i, col_nb_bv]).upper():
            # Ajouter i, i+1, i+2
            idx_supprimer.extend(range(i, min(i+2, len(df_copy))))
            i += 2  # Sauter les 3 lignes
        else:
            i += 1
    
    # Supprimer
    if idx_supprimer:
        df_copy = df_copy.drop(idx_supprimer).reset_index(drop=True)
        print(f"Supprimé {len(idx_supprimer)} lignes")
    # Supprimer la colonne N° REGION
    df_copy = df_copy.drop(columns=['N° REGION'], errors='ignore')
    
    df_copy["region"] = df_copy["region"].str.replace("\n", "")
    # dans la colone REGION remplaçons REGION par ""
    df_copy["region"] = df_copy["region"].str.replace("REGION", "")

    for col in ["taux_de_participation","pourcentage_blancs","pourcentage_voix"] :
        df_copy[col] = df_copy[col].str.replace("%", "")
        df_copy[col] = df_copy[col].str.replace(",", ".")
        #convertion des en type float
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce').fillna(0).astype(float)


    for col in ["bulletins_blancs","inscrits","suffrages_exprimes","voix","votants","nombre_bureaux_vote","bulletins_nuls"] :
        df_copy[col] = df_copy[col].str.replace(" ", "")
    
    # Remplacer les chaine vide de la colone REGION par le nom de la region precedente
    df_copy["region"] = df_copy["region"].replace("", method="ffill")

    # suprimer les "" dans la colone circonscription
    df_copy["circonscription"] = df_copy["circonscription"].replace("", method="ffill")
        
    # convertissons ses colonne en entier
    int_columns = ["bulletins_blancs", "bulletins_nuls", "suffrages_exprimes", "inscrits", "votants", "nombre_bureaux_vote","voix"]
    for col in int_columns:
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce').fillna(0).astype(int)
    
    # Suprimons la premiere ligne du dataset
    df_copy = df_copy.drop(index=0).reset_index(drop=True)
    
    return df_copy

dataset = nettoyer(df)
print(len(dataset))
dataset.head(100)

Supprimé 70 lignes
1125


C:\Users\MSI\AppData\Local\Temp\ipykernel_19404\3971967876.py:48: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df_copy["region"] = df_copy["region"].replace("", method="ffill")
C:\Users\MSI\AppData\Local\Temp\ipykernel_19404\3971967876.py:51: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df_copy["circonscription"] = df_copy["circonscription"].replace("", method="ffill")


,region,circonscription,nombre_bureaux_vote,inscrits,votants,taux_de_participation,bulletins_nuls,suffrages_exprimes,bulletins_blancs,pourcentage_blancs,parti,candidat,voix,pourcentage_voix,elu
0,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, \nL...",144,52106,14070,27.00,388,13682,76,0.56,INDEPENDANT,KOTO EHOU SOPIE,547,4.00,
1,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, \nL...",0,0,0,0.00,0,0,0,0.00,RHDP,KOFFI AKA CHARLES,9078,66.35,ELU(E)
2,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, \nL...",0,0,0,0.00,0,0,0,0.00,INDEPENDANT,YEPIE KOUASSI THEODORE,58,0.42,
3,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, \nL...",0,0,0,0.00,0,0,0,0.00,INDEPENDANT,TCHIMOU GNAMON BERTRAND,1991,14.55,
4,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, \nL...",0,0,0,0.00,0,0,0,0.00,INDEPENDANT,BOKA BOKA MAURICE,674,4.93,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,BELIER,"LOMOKANKRO, MOLONOU, TLEBISSOU ET YAKPABO-\nSA...",147,42781,14778,34.54,537,14241,134,0.94,INDEPENDANT,ENSEMBLE TOUT EST POSSIBLE,80,0.56,
96,BELIER,"LOMOKANKRO, MOLONOU, TLEBISSOU ET YAKPABO-\nSA...",0,0,0,0.00,0,0,0,0.00,INDEPENDANT,LES PROGRESSISTES,222,1.56,
97,BELIER,"LOMOKANKRO, MOLONOU, TLEBISSOU ET YAKPABO-\nSA...",0,0,0,0.00,0,0,0,0.00,RHDP,"UNE COTE DIVOIRE EN PAIX , PROSPERE ET SOLIDAIRE",6127,43.02,
98,BELIER,"LOMOKANKRO, MOLONOU, TLEBISSOU ET YAKPABO-\nSA...",0,0,0,0.00,0,0,0,0.00,INDEPENDANT,ENSEMBLE POUR LE DEVELOPPEMENT,190,1.33,


In [9]:
dataset.dtypes

region                    object
circonscription           object
nombre_bureaux_vote        int64
inscrits                   int64
votants                    int64
taux_de_participation    float64
bulletins_nuls             int64
suffrages_exprimes         int64
bulletins_blancs           int64
pourcentage_blancs       float64
parti                     object
candidat                  object
voix                       int64
pourcentage_voix         float64
elu                       object
dtype: object

##### ***Gestion des valeurs manquantes pour les colonnes clés***

In [ ]:
df_clean = dataset.copy()

print("\n Aperçu des données avant remplissage :")    
for col in ['region', 'circonscription', 'parti', 'candidat']:
    missing = df_clean[col].isna().sum()
    total = len(df_clean)
    print(f"   {col}: {missing} valeurs manquantes ({missing/total*100:.1f}%)")



 Aperçu des données avant remplissage :
   region: 0 valeurs manquantes (0.0%)
   circonscription: 0 valeurs manquantes (0.0%)
   parti: 0 valeurs manquantes (0.0%)
   candidat: 0 valeurs manquantes (0.0%)


##### ***Normalisation des chaînes de caractères***

In [11]:
import re

def normalize_text(text):
    if pd.isna(text):
        return text
    # Convertir en string
    text = str(text)
    # Supprimer les sauts de ligne multiples
    text = re.sub(r'\n+', ' ', text)
    # Remplacer les espaces multiples
    text = re.sub(r'\s+', ' ', text)
    # Supprimer les espaces en début/fin
    return text.strip()

# Application aux colonnes textuelles
text_columns = ['region', 'circonscription', 'parti', 'candidat', 'elu']
for col in text_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].apply(normalize_text)
        print(f"   ✓ Colonne '{col}' normalisée")

   ✓ Colonne 'region' normalisée
   ✓ Colonne 'circonscription' normalisée
   ✓ Colonne 'parti' normalisée
   ✓ Colonne 'candidat' normalisée
   ✓ Colonne 'elu' normalisée


##### ***Normalisation avancée (pour la recherche robuste)***

In [12]:
print("\n Création des colonnes normalisées...")

def create_slug(text):

    if pd.isna(text):
        return ""
    text = str(text)
    # Normaliser les caractères spéciaux
    text = text.upper()
    # Remplacer les caractères accentués
    replacements = {
        'É': 'E', 'È': 'E', 'Ê': 'E', 'Ë': 'E',
        'À': 'A', 'Â': 'A', 'Ä': 'A',
        'Î': 'I', 'Ï': 'I',
        'Ô': 'O', 'Ö': 'O',
        'Ù': 'U', 'Û': 'U', 'Ü': 'U',
        'Ç': 'C', "'": '', '-': '_', ' ': '_'
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    # Supprimer les caractères non alphanumériques
    text = re.sub(r'[^A-Z0-9_]', '', text)
    # Convertir en minuscules
    return text.lower()

# Colonnes normalisées
df_clean['region_normalized'] = df_clean['region'].apply(lambda x: str(x).upper() if pd.notna(x) else '')
df_clean['region_slug'] = df_clean['region_normalized'].apply(create_slug)

df_clean['parti_normalized'] = df_clean['parti'].apply(lambda x: str(x).upper() if pd.notna(x) else '')
df_clean['parti_slug'] = df_clean['parti_normalized'].apply(create_slug)

df_clean['candidat_normalized'] = df_clean['candidat'].apply(lambda x: str(x).upper() if pd.notna(x) else '')

# Pour circonscription : créer un ID unique
def create_circonscription_id(row):
    """Crée un ID unique pour une circonscription"""
    region = str(row['region_slug']) if pd.notna(row['region_slug']) else ''
    circ = str(row['circonscription']) if pd.notna(row['circonscription']) else ''
    
    # Prendre les 3-4 premiers mots pour l'ID
    words = circ.split()[:4]
    circ_slug = '_'.join(words).lower()
    circ_slug = re.sub(r'[^a-z0-9_]', '', circ_slug)
    
    return f"{region}_{circ_slug}" if region and circ_slug else ''

# Appliquer la fonction
df_clean['circonscription_id'] = df_clean.apply(create_circonscription_id, axis=1)
df_clean['circonscription_normalized'] = df_clean['circonscription'].apply(
    lambda x: str(x).upper().replace('\n', ' ') if pd.notna(x) else ''
)


 Création des colonnes normalisées...


In [13]:
df_clean.head(10)

,region,circonscription,nombre_bureaux_vote,inscrits,votants,taux_de_participation,bulletins_nuls,suffrages_exprimes,bulletins_blancs,pourcentage_blancs,...,voix,pourcentage_voix,elu,region_normalized,region_slug,parti_normalized,parti_slug,candidat_normalized,circonscription_id,circonscription_normalized
0,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",144,52106,14070,27.00,388,13682,76,0.56,...,547,4.00,,AGNEBY-TIASSA,agneby_tiassa,INDEPENDANT,independant,KOTO EHOU SOPIE,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV..."
1,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,9078,66.35,ELU(E),AGNEBY-TIASSA,agneby_tiassa,RHDP,rhdp,KOFFI AKA CHARLES,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV..."
2,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,58,0.42,,AGNEBY-TIASSA,agneby_tiassa,INDEPENDANT,independant,YEPIE KOUASSI THEODORE,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV..."
3,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,1991,14.55,,AGNEBY-TIASSA,agneby_tiassa,INDEPENDANT,independant,TCHIMOU GNAMON BERTRAND,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV..."
4,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,674,4.93,,AGNEBY-TIASSA,agneby_tiassa,INDEPENDANT,independant,BOKA BOKA MAURICE,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV..."
5,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,331,2.42,,AGNEBY-TIASSA,agneby_tiassa,ADCI,adci,EDI DOFFOU PAUL,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV..."
6,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,474,3.46,,AGNEBY-TIASSA,agneby_tiassa,FPI,fpi,N'GUESSAN KOTCHI REMI,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV..."
7,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,453,3.31,,AGNEBY-TIASSA,agneby_tiassa,MGC,mgc,NANDO YAVO SERGE,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV..."
8,AGNEBY-TIASSA,AGBOVILLE COMMUNE,133,48710,12821,26.32,317,12504,81,0.65,...,296,2.37,,AGNEBY-TIASSA,agneby_tiassa,ADCI,adci,OCHOU WROHOUM MARIE-PASCALE,agneby_tiassa_agboville_commune,AGBOVILLE COMMUNE
9,AGNEBY-TIASSA,AGBOVILLE COMMUNE,0,0,0,0.00,0,0,0,0.00,...,10675,85.37,ELU(E),AGNEBY-TIASSA,agneby_tiassa,RHDP,rhdp,DIMBA N'GOU PIERRE,agneby_tiassa_agboville_commune,AGBOVILLE COMMUNE


##### ***Standardisation des partis (regroupement des variantes)***

In [14]:
print("\n Standardisation des noms de partis...")

# Dictionnaire de standardisation (à compléter selon vos besoins)
parti_standardization = {
    'RHDP': 'RHDP',
    'R.H.D.P': 'RHDP',
    'RASSEMBLEMENT DES HOUPHOUËTISTES': 'RHDP',
    'PDCI-RDA': 'PDCI-RDA',
    'PDCI': 'PDCI-RDA',
    'FPI': 'FPI',
    'INDEPENDANT': 'INDEPENDANT',
    'INDÉPENDANT': 'INDEPENDANT',
    'INDEP': 'INDEPENDANT',
}

def standardize_parti(parti):
    if pd.isna(parti):
        return 'INDEPENDANT'
    parti_str = str(parti).upper().strip()
    # Chercher une correspondance exacte
    if parti_str in parti_standardization:
        return parti_standardization[parti_str]
    # Chercher une correspondance partielle
    for key, value in parti_standardization.items():
        if key in parti_str or parti_str in key:
            return value
    return parti_str  # Retourner l'original si non trouvé

df_clean['parti_standardized'] = df_clean['parti_normalized'].apply(standardize_parti)

df_clean.head(10)


 Standardisation des noms de partis...


,region,circonscription,nombre_bureaux_vote,inscrits,votants,taux_de_participation,bulletins_nuls,suffrages_exprimes,bulletins_blancs,pourcentage_blancs,...,pourcentage_voix,elu,region_normalized,region_slug,parti_normalized,parti_slug,candidat_normalized,circonscription_id,circonscription_normalized,parti_standardized
0,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",144,52106,14070,27.00,388,13682,76,0.56,...,4.00,,AGNEBY-TIASSA,agneby_tiassa,INDEPENDANT,independant,KOTO EHOU SOPIE,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",INDEPENDANT
1,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,66.35,ELU(E),AGNEBY-TIASSA,agneby_tiassa,RHDP,rhdp,KOFFI AKA CHARLES,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",RHDP
2,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,0.42,,AGNEBY-TIASSA,agneby_tiassa,INDEPENDANT,independant,YEPIE KOUASSI THEODORE,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",INDEPENDANT
3,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,14.55,,AGNEBY-TIASSA,agneby_tiassa,INDEPENDANT,independant,TCHIMOU GNAMON BERTRAND,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",INDEPENDANT
4,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,4.93,,AGNEBY-TIASSA,agneby_tiassa,INDEPENDANT,independant,BOKA BOKA MAURICE,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",INDEPENDANT
5,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,2.42,,AGNEBY-TIASSA,agneby_tiassa,ADCI,adci,EDI DOFFOU PAUL,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",ADCI
6,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,3.46,,AGNEBY-TIASSA,agneby_tiassa,FPI,fpi,N'GUESSAN KOTCHI REMI,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",FPI
7,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",0,0,0,0.00,0,0,0,0.00,...,3.31,,AGNEBY-TIASSA,agneby_tiassa,MGC,mgc,NANDO YAVO SERGE,agneby_tiassa_aboude_attobrou_guessiguie_grand...,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",MGC
8,AGNEBY-TIASSA,AGBOVILLE COMMUNE,133,48710,12821,26.32,317,12504,81,0.65,...,2.37,,AGNEBY-TIASSA,agneby_tiassa,ADCI,adci,OCHOU WROHOUM MARIE-PASCALE,agneby_tiassa_agboville_commune,AGBOVILLE COMMUNE,ADCI
9,AGNEBY-TIASSA,AGBOVILLE COMMUNE,0,0,0,0.00,0,0,0,0.00,...,85.37,ELU(E),AGNEBY-TIASSA,agneby_tiassa,RHDP,rhdp,DIMBA N'GOU PIERRE,agneby_tiassa_agboville_commune,AGBOVILLE COMMUNE,RHDP


##### ***Colonnes calculées***

In [22]:
from datetime import datetime
# 5. Colonnes calculées
print("\n Calcul des colonnes supplémentaires...")

# Convertir 'elu' en indicateur binaire
df_clean['est_elu'] = df_clean['elu'].apply(
    lambda x: 1 if pd.notna(x) and 'ELU' in str(x).upper() else 0
)

# # Timestamp de traitement
df_clean['processed_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# ID de ligne unique
df_clean['row_id'] = range(1, len(df_clean) + 1)

print(f"\n Nettoyage terminé !")
print(f"   Colonnes finales : {list(df_clean.columns)}")
print(f"\n Aperçu des données nettoyées :")
df_clean[['row_id', 'region', 'circonscription', 'parti_standardized', 'candidat', 'voix', 'est_elu']].head(10)


 Calcul des colonnes supplémentaires...

 Nettoyage terminé !
   Colonnes finales : ['region', 'circonscription', 'nombre_bureaux_vote', 'inscrits', 'votants', 'taux_de_participation', 'bulletins_nuls', 'suffrages_exprimes', 'bulletins_blancs', 'pourcentage_blancs', 'parti', 'candidat', 'voix', 'pourcentage_voix', 'elu', 'region_normalized', 'region_slug', 'parti_normalized', 'parti_slug', 'candidat_normalized', 'circonscription_id', 'circonscription_normalized', 'parti_standardized', 'est_elu', 'row_id', 'processed_at']

 Aperçu des données nettoyées :


,row_id,region,circonscription,parti_standardized,candidat,voix,est_elu
0,1,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",INDEPENDANT,KOTO EHOU SOPIE,547,0
1,2,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",RHDP,KOFFI AKA CHARLES,9078,1
2,3,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",INDEPENDANT,YEPIE KOUASSI THEODORE,58,0
3,4,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",INDEPENDANT,TCHIMOU GNAMON BERTRAND,1991,0
4,5,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",INDEPENDANT,BOKA BOKA MAURICE,674,0
5,6,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",ADCI,EDI DOFFOU PAUL,331,0
6,7,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",FPI,N'GUESSAN KOTCHI REMI,474,0
7,8,AGNEBY-TIASSA,"ABOUDE, ATTOBROU, GUESSIGUIE, GRAND-MORIÉ, LOV...",MGC,NANDO YAVO SERGE,453,0
8,9,AGNEBY-TIASSA,AGBOVILLE COMMUNE,ADCI,OCHOU WROHOUM MARIE-PASCALE,296,0
9,10,AGNEBY-TIASSA,AGBOVILLE COMMUNE,RHDP,DIMBA N'GOU PIERRE,10675,1


### ***Quelques verifications***

In [23]:
# le nombre de candidats unique dans le dataset
print("le nombre de candidats unique dans le dataset :", len(df_clean["candidat"].unique()))
# le nombre de candidats dans le dataset
print("le nombre de candidats dans le dataset :", len(df_clean["candidat"]))
# le nombre de bureaux de vote 
print("le nombre de bureaux de vote :", df_clean["nombre_bureaux_vote"].sum())
# le nombre d'inscrit 
print("le nombre d'inscrit :", df_clean["inscrits"].sum())
# le nombre de buletins blanc 
print("le nombre de buletins blanc :", df_clean["bulletins_blancs"].sum())
# le nombre de circonscription 
print("le nombre de circonscription :", len(df_clean["circonscription_id"].unique()))
# le nombre de region
print("le nombre de region unique :", len(df_clean["region"].unique()))
# le nombre de parti
print("le nombre de parti unique :", len(df_clean["parti"].unique()))
#statistiques des élus
print("stat d'élus :", df_clean["est_elu"].value_counts())
#le nombre de voix par region
print("le nombre de voix par region :", df_clean.groupby("region")["voix"].sum())
#le nombre de surfrage exprimé par region
print("le nombre de surfrage exprimé par region :", df_clean.groupby("region")["suffrages_exprimes"].sum())
#le pourcentage total de bulettins blancs par region
print("le pourcentage total de bulettins blancs par region :", df_clean.groupby("region")["pourcentage_blancs"].sum())

le nombre de candidats unique dans le dataset : 1074
le nombre de candidats dans le dataset : 1125
le nombre de bureaux de vote : 25338
le nombre d'inscrit : 8597092
le nombre de buletins blanc : 29578
le nombre de circonscription : 205
le nombre de region unique : 33
le nombre de parti unique : 43
stat d'élus : est_elu
0    920
1    205
Name: count, dtype: int64
le nombre de voix par region : region
AGNEBY-TIASSA                         80937
BAFING                                53051
BAGOUE                               102278
BELIER                                46563
BERE                                  77373
BOUNKANI                              36277
CAVALLY                               41644
DISTRICT AUTONOME D'ABIDJAN          422006
DISTRICT AUTONOME DE YAMOUSSOUKRO     47584
FOLON                                 33465
GBEKE                                177750
GBOKLE                                20279
GOH                                  122612
GONTOUGO                

In [24]:

def sauvegarder_par_region(dataset):
        
    blocks = []
    current_region = ""
    current_data = []
    
    for i in range(len(dataset)):
        row = dataset.iloc[i]
        region_val = str(row["region"]).strip() if pd.notna(row["region"]) else ""
        
        # Si nouvelle région non vide
        if region_val and region_val != current_region:
            # Sauvegarder bloc précédent
            if current_data:
                blocks.append((current_region, pd.DataFrame(current_data)))
            
            # Nouveau bloc
            current_region = region_val
            current_data = [row]
        else:
            # Ajouter au bloc courant
            current_data.append(row)
    
    # Dernier bloc
    if current_data:
        blocks.append((current_region, pd.DataFrame(current_data)))
    
    # Sauvegarde
    for region_name, df in blocks:
        # Nom de fichier simple
        filename = f"region_{region_name[:50]}.csv".replace("\n", "_").replace(" ", "_")
        df.to_csv(f"E:\\Projet_artefact\\src\\data\\regions_csv\\{filename}", index=False, encoding='utf-8')
        print(f"Sauvegardé: {filename} ({len(df)} lignes)")
    
    return blocks

# Utilisation
blocs = sauvegarder_par_region(df_clean)

Sauvegardé: region_AGNEBY-TIASSA.csv (60 lignes)
Sauvegardé: region_BAFING.csv (18 lignes)
Sauvegardé: region_BAGOUE.csv (14 lignes)
Sauvegardé: region_BELIER.csv (19 lignes)
Sauvegardé: region_BERE.csv (20 lignes)
Sauvegardé: region_BOUNKANI.csv (20 lignes)
Sauvegardé: region_CAVALLY.csv (23 lignes)
Sauvegardé: region_DISTRICT_AUTONOME_D'ABIDJAN.csv (92 lignes)
Sauvegardé: region_DISTRICT_AUTONOME_DE_YAMOUSSOUKRO.csv (23 lignes)
Sauvegardé: region_FOLON.csv (7 lignes)
Sauvegardé: region_GBEKE.csv (47 lignes)
Sauvegardé: region_GBOKLE.csv (14 lignes)
Sauvegardé: region_GOH.csv (87 lignes)
Sauvegardé: region_GONTOUGO.csv (28 lignes)
Sauvegardé: region_GRANDS_PONTS.csv (16 lignes)
Sauvegardé: region_GUEMON.csv (52 lignes)
Sauvegardé: region_HAMBOL.csv (16 lignes)
Sauvegardé: region_HAUT-_SASSANDRA.csv (59 lignes)
Sauvegardé: region_IFFOU.csv (23 lignes)
Sauvegardé: region_INDENIE-DJUABLIN.csv (22 lignes)
Sauvegardé: region_KABADOUGOU.csv (25 lignes)
Sauvegardé: region_LA_ME.csv (42 ligne